In [1]:
import pandas as pd
import backtrader as bt
from datetime import datetime
from loguru import logger

In [5]:
class CrossOverAtrStop(bt.Strategy):
    # параметры для скользящих средних
    params = dict(
        fast_ma=10,
        slow_ma=20,
        period=3,
        multiplier=3
    )

    def __init__(self):
        '''рассчёт скользящих средних и индикатора их пересечения'''
        fast_ma = bt.ind.SMA(period=self.p.fast_ma)
        slow_ma = bt.ind.SMA(period=self.p.slow_ma)
        self.crossover = bt.ind.CrossOver(fast_ma, slow_ma)
        self.atr = bt.ind.ATR(period=self.p.period)
        
        self.orderl = list() # для хранения состояния заявки

    def notify_order(self, order):
        '''вывод сделок в консоль'''
        if order.status in [order.Submitted, order.Accepted]:
            return
        
        if order.status in [order.Completed]:
            if order.isbuy():
                logger.info(f'BUY@price {order.executed.price}')
            elif order.issell():
                logger.info(f'SELL@price {order.executed.price}')
        elif order.status in [order.Margin, order.Rejected]:
            logger.warning('Order Margin/Rejected')

        self.order = None

    def next(self):
        if not self.position and self.crossover > 0:
                stop_price = self.data.close[0] - self.atr * self.p.multiplier
                o1 = self.buy(transmit=False) # основная зявки
                o2 = self.sell(exectype=bt.Order.Stop, price=stop_price, parent=o1, transmit=True) # стоп
                self.orderl = [o1, o2]
        elif self.crossover < 0:
            for o in self.orderl: 
                self.cancel(o)
            self.close()


In [6]:
cerebro = bt.Cerebro()
commission = 0.05
cerebro.broker.setcommission(commission=commission/100, leverage=1)
cerebro.broker.setcash(10000.) # стартовый кэш
cerebro.addsizer(bt.sizers.PercentSizer, percents=95) # размер позиции

df = pd.read_csv(
    '..//data//1d//AAPL.csv', # файл с котировками
    index_col=0,
    parse_dates=True
)

data = bt.feeds.PandasData(
    dataname=df,
    openinterest=-1,
    fromdate=datetime(2005, 1, 1), # стартовая дата
    # todate=datetime(2005, 2, 1),
    timeframe=bt.TimeFrame.Days,
    compression=1
)

cerebro.adddata(data) # добавляем котировки
cerebro.addstrategy(CrossOverAtrStop,fast_ma=10,slow_ma=20,period=3,multiplier=3) # добавляем стратегию
cerebro.run() # запускаем рассчёты

2021-10-19 11:05:18.268 | INFO     | __main__:notify_order:26 - BUY@price 1.5199999809265134
2021-10-19 11:05:18.275 | INFO     | __main__:notify_order:28 - SELL@price 1.5175000429153442
2021-10-19 11:05:18.294 | INFO     | __main__:notify_order:26 - BUY@price 1.264999985694885
2021-10-19 11:05:18.300 | INFO     | __main__:notify_order:28 - SELL@price 1.2342859506607056
2021-10-19 11:05:18.311 | INFO     | __main__:notify_order:26 - BUY@price 1.4107140302658079
2021-10-19 11:05:18.325 | INFO     | __main__:notify_order:28 - SELL@price 1.2828569412231443
2021-10-19 11:05:18.344 | INFO     | __main__:notify_order:26 - BUY@price 1.3296430110931394
2021-10-19 11:05:18.356 | INFO     | __main__:notify_order:28 - SELL@price 1.3703570365905762
2021-10-19 11:05:18.366 | INFO     | __main__:notify_order:26 - BUY@price 1.4632140398025513
2021-10-19 11:05:18.388 | INFO     | __main__:notify_order:28 - SELL@price 1.5521429777145386
2021-10-19 11:05:18.394 | INFO     | __main__:notify_order:26 - BU

In [7]:
cerebro.plot(iplot=False) # рисуем графики

[[<Figure size 640x480 with 6 Axes>]]